# Автоматический парсинг объявлений Avito

## Цель

Построить универсальный пайплайн для сбора и структурирования объявлений Avito на основе HTML-страниц.  
Результат — датасет, пригодный для задач машинного обучения (текст + изображения).

---

## Пайплайн

1. Ручное сохранение HTML-страниц поисковой выдачи Avito  
2. Извлечение ссылок на объявления  
3. Загрузка HTML объявлений через headless-браузер  
4. Парсинг HTML объявлений  
5. Извлечение и сохранение изображений  

---

## Парсинг объявлений

Из HTML объявления извлекаются:
- заголовок;
- описание (сырое и нормализованное);
- категории товара;
- характеристики;
- изображения.

Описание извлекается с учётом разных версий верстки Avito (fallback по DOM).  
Текст нормализуется и очищается от псевдокириллицы.

---

## Категории и характеристики

Категории извлекаются из breadcrumbs позиционно.  
Характеристики извлекаются по DOM-паттерну «название → значение» и автоматически расширяются для разных категорий товаров.

---

## Изображения

Ссылки на изображения извлекаются из клиентского состояния страницы.  
Изображения фильтруются по качеству, дедуплицируются и сохраняются локально.

---

## Результат

Формируется структурированный датасет объявлений и набор изображений.  
Весь процесс воспроизводим и выполняется офлайн.

---

**Окружение:**  
Python 3.11 (локальный запуск), версии библиотек зафиксированы в `requirements.txt`.

## Импорт библиотек и настройка директорий

На этом этапе подключаются все необходимые библиотеки: для работы с HTML, изображениями, веб-браузером и файловой системой.  
Также определяются директории, в которых будут храниться HTML-страницы объявлений, изображения и ссылки.

In [13]:
import re
import json
from tqdm import tqdm
import time
import urllib.parse
from pathlib import Path
from typing import Set
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from PIL import Image
from io import BytesIO
import requests
import imagehash

# Проверка окружения 
import sys
print("Python:", sys.version)

# --- Папки ---
CATEGORY = "top_woman"
BASE_DIR = Path("data")
HTML_DIR = BASE_DIR / "html" / CATEGORY / "ads"
HTML_LIST_DIR = BASE_DIR / "html" / CATEGORY / "list"
IMG_DIR = BASE_DIR / "images" / CATEGORY

LINKS_DIR = BASE_DIR / "links"
DATASETS_DIR = BASE_DIR / "datasets"
LINKS_PATH = BASE_DIR / "links" / f"links_{CATEGORY}.txt"
OUT_JSONL = BASE_DIR / "datasets" / f"dataset_{CATEGORY}.jsonl"

HTML_LIST_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)
LINKS_DIR.mkdir(parents=True, exist_ok=True)
DATASETS_DIR.mkdir(parents=True, exist_ok=True)

Python: 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]


## Извлечение ссылок на объявления из HTML списка

Данный блок предназначен для извлечения ссылок на отдельные объявления из HTML-страницы со списком товаров (результаты поиска или категории).

### Логика работы

1. HTML страницы парсится с помощью BeautifulSoup.
2. По атрибуту `data-marker="item"` находятся контейнеры отдельных карточек объявлений.
3. Внутри каждой карточки извлекается ссылка `<a>` с атрибутом `href`.
4. Из ссылки удаляются GET-параметры.
5. С помощью регулярного выражения проверяется наличие идентификатора объявления в конце URL.
6. Корректные ссылки приводятся к абсолютному виду и добавляются в множество.

### Результат

Функция возвращает множество уникальных ссылок на объявления, которые далее используются для загрузки HTML-страниц отдельных товаров.

In [2]:
AD_ID_PATTERN = re.compile(r"_(\d+)$")

def extract_ad_links_from_list_html(html_text: str):
    soup = BeautifulSoup(html_text, "html.parser")
    links = set()

    for item in soup.find_all("div", attrs={"data-marker": "item"}):
        a = item.find("a", href=True)
        if not a:
            continue

        href = a["href"].split("?")[0]
        if AD_ID_PATTERN.search(href):
            links.add("https://www.avito.ru" + href)

    return links

## Сбор ссылок на объявления из HTML-страниц списков

В данном блоке выполняется пакетная обработка HTML-файлов со списками объявлений.

HTML-файлы со списками представляют собой **вручную сохранённые страницы поисковой выдачи Avito для выбранной категории товаров**. Такой подход позволяет работать с локальными HTML без обращения к API и без дополнительных сетевых запросов.

Для каждой HTML-страницы:
- считывается содержимое файла;
- извлекаются ссылки на отдельные объявления с помощью функции `extract_ad_links_from_list_html`;
- ссылки агрегируются в общее множество для устранения дубликатов.

После обработки всех файлов:
- ссылки сортируются;
- сохраняются в текстовый файл для дальнейшего использования в пайплайне.

### Результат

Формируется файл со списком уникальных ссылок на объявления выбранной категории, который используется на следующем этапе загрузки HTML отдельных товаров.

In [3]:
list_files = sorted(HTML_LIST_DIR.glob("*.html"))
print("List pages:", len(list_files))

all_links = set()
for f in tqdm(list_files):
    html_text = f.read_text(encoding="utf-8", errors="ignore")
    all_links |= extract_ad_links_from_list_html(html_text)

all_links = sorted(all_links)

print("Total unique ads:", len(all_links))
print("Sample:", all_links[:5])

LINKS_PATH.write_text("\n".join(all_links), encoding="utf-8")
print("Saved to:", LINKS_PATH)

List pages: 6


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:07<00:00,  1.20s/it]

Total unique ads: 379
Sample: ['https://www.avito.ru/agalatovo/odezhda_obuv_aksessuary/naryadnye_bodi_intimissimi_42_7724048899', 'https://www.avito.ru/anapa/odezhda_obuv_aksessuary/novye_futbolki_barbara_ilvisi_italiya_razmer_s_7598965573', 'https://www.avito.ru/arhangelsk/odezhda_obuv_aksessuary/mayka_top_liu_jo_7832820310', 'https://www.avito.ru/balakovo/odezhda_obuv_aksessuary/svitshot_oversayz_novyy_s_birkoy_razmer_42-50_4381479867', 'https://www.avito.ru/balashiha/odezhda_obuv_aksessuary/futbolka_trikotto_i_bryuki_lavin_raznye_razmery_7427330150']
Saved to: data\links_top_woman.txt


## Настройка headless-браузера для загрузки HTML

В данном блоке настраивается headless-браузер Chrome для автоматизированной загрузки HTML-страниц Avito.

Используется режим без графического интерфейса (headless), что позволяет:
- запускать браузер в средах без GUI;
- ускорить загрузку страниц;
- снизить потребление ресурсов.

### Основные параметры

- `--headless=new` — использование нового headless-режима Chrome;
- `--disable-gpu` — отключение GPU для стабильной работы;
- `--no-sandbox` — необходим в изолированных средах;
- `--window-size=1920,1080` — фиксированный размер окна для корректной загрузки элементов страницы.

Браузер используется далее для получения и сохранения HTML-страниц объявлений и поисковой выдачи.

In [4]:
# Настройка headless-браузера
chrome_options = Options()
chrome_options.add_argument("--headless=new")  # Новый headless-режим
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

## Загрузка и сохранение HTML-страницы Avito

Данная функция предназначена для загрузки HTML-страницы Avito с помощью headless-браузера и сохранения её локально.

### Логика работы

1. Браузер переходит по заданному URL.
2. Выполняется пауза для завершения загрузки JavaScript-контента и формирования клиентского состояния страницы.
3. Полученный HTML-код страницы сохраняется в файл без дополнительной обработки.

Такой подход позволяет зафиксировать HTML-страницу в том виде, в котором она используется браузером, и далее выполнять парсинг локально, без повторных сетевых запросов.

In [5]:
def fetch_and_save_avito_page(url: str, dest_path: Path):
    driver.get(url)
    time.sleep(5)  # дать JS прогрузить JSON
    html = driver.page_source
    dest_path.write_text(html, encoding="utf-8")

## Загрузка HTML-страниц отдельных объявлений

В данном блоке выполняется загрузка HTML-страниц отдельных объявлений Avito по заранее собранному списку ссылок.

### Логика работы

1. Из текстового файла загружается список URL объявлений.
2. Для каждого URL извлекается идентификатор объявления (`ad_id`).
3. Проверяется наличие соответствующего HTML-файла локально.
4. Если файл отсутствует:
   - страница загружается с помощью headless-браузера;
   - HTML сохраняется в файл;
   - выполняется пауза между запросами.

### Назначение

Такой подход:
- предотвращает повторную загрузку уже сохранённых страниц, предупреждая бан ip;
- снижает нагрузку на сайт;
- позволяет далее выполнять парсинг полностью офлайн, работая только с локальными HTML-файлами.

In [6]:
def load_avito_links(path: Path) -> list:
    with path.open(encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]
        
avito_urls = load_avito_links(LINKS_PATH)

for url in tqdm(avito_urls):
    ad_id = url.split('_')[-1].split("?")[0]
    html_path = HTML_DIR / f"{ad_id}.html"

    if not html_path.exists():
        fetch_and_save_avito_page(url, html_path)
        time.sleep(4)  # пауза

100%|██████████████████████████████████████████████████████████████████████████████| 379/379 [1:38:36<00:00, 15.61s/it]


## Извлечение ссылок на изображения объявления

Данный блок предназначен для извлечения URL изображений из HTML-файла объявления.

Изображения на Avito не всегда представлены напрямую в DOM.  
Поэтому используется клиентское состояние страницы `window.__preloadedState__`, в котором содержатся все метаданные объявления, включая ссылки на изображения.

### Логика работы

1. HTML-файл парсится с помощью BeautifulSoup.
2. Внутри `<script>`-тегов ищется блок с `window.__preloadedState__`.
3. Содержимое состояния:
   - декодируется из URL-encoded строки;
   - преобразуется в JSON.
4. JSON обходится рекурсивно, извлекая все строки, содержащие ссылки на изображения Avito (`img.avito.st/image`).

### Результат

Функция возвращает множество (`set`) URL изображений объявления.  
Дальнейшая загрузка, фильтрация по качеству и дедупликация изображений выполняются в отдельных функциях.

In [7]:
def extract_image_urls_from_html_file(html_path: Path) -> Set[str]:
    soup = BeautifulSoup(html_path.read_text(encoding="utf-8", errors="ignore"), "html.parser")
    script_text = None

    for s in soup.find_all("script"):
        if s.string and "window.__preloadedState__" in s.string:
            script_text = s.string
            break

    if not script_text:
        return set()

    m = re.search(r'window\.__preloadedState__\s*=\s*"(.+?)";', script_text)
    if not m:
        return set()

    decoded = urllib.parse.unquote(m.group(1))
    try:
        data = json.loads(decoded)
    except:
        return set()

    urls = set()
    def walk(obj):
        if isinstance(obj, dict):
            for v in obj.values(): walk(v)
        elif isinstance(obj, list):
            for v in obj: walk(v)
        elif isinstance(obj, str) and "img.avito.st/image" in obj:
            urls.add(obj)

    walk(data)
    return urls

## Загрузка и отбор изображений объявления

Данный блок отвечает за загрузку изображений по URL и отбор качественных, уникальных фотографий объявления.

### Загрузка изображений

Функция `download_image`:
- загружает изображение по URL;
- открывает его с помощью PIL;
- приводит к формату RGB;
- в случае ошибки возвращает `None`, не прерывая пайплайн.

Это обеспечивает устойчивую работу при битых ссылках или сетевых сбоях.

### Фильтрация и дедупликация изображений

Функция `select_high_quality_images` выполняет несколько этапов обработки:

1. **Фильтрация по качеству**  
   Оставляются только изображения, у которых минимальная сторона не меньше `min_side` пикселей.

2. **Поиск дубликатов**  
   Для каждого изображения вычисляется perceptual hash (pHash).  
   Изображения группируются, если расстояние между хэшами не превышает порог `hash_thresh`.

3. **Выбор лучшего изображения из группы**  
   В каждой группе дубликатов сохраняется изображение с максимальным разрешением.

### Результат

Функция возвращает список уникальных изображений высокого качества, которые далее используются в датасете и мультимодальных моделях.

In [8]:
def download_image(url):
    try:
        r = requests.get(url, timeout=10)
        img = Image.open(BytesIO(r.content)).convert("RGB")
        return img
    except:
        return None

def select_high_quality_images(urls, min_side=400, hash_thresh=5):
    imgs = []
    for u in urls:
        img = download_image(u)
        if img and min(img.size) >= min_side:
            imgs.append((u, img))

    groups = []
    for url, img in imgs:
        h = imagehash.phash(img)
        for g in groups:
            if abs(h - g[0][2]) <= hash_thresh:
                g.append((url, img, h))
                break
        else:
            groups.append([(url, img, h)])

    final = []
    for g in groups:
        best = max(g, key=lambda x: x[1].size[0] * x[1].size[1])
        final.append((best[0], best[1]))

    return final

## Нормализация текста описаний объявлений
### Задача

Текстовые описания объявлений, выгруженные из HTML Avito, часто содержат так называемую псевдокириллицу: слова на русском языке, в которых отдельные буквы заменены визуально похожими латинскими символами.
Пример: СОБCTBEHHОЕ, PАСЦВЕTОK, ПРОИЗBОДCТВО.

Такая подмена:
* ухудшает качество текстовых признаков,
* мешает токенизации,
* приводит к разрыву семантически одинаковых слов.

При этом в объявлениях могут встречаться настоящие английские слова (бренды, названия моделей), которые нельзя «насильно» переводить в кириллицу.

### Принцип решения

Используется контекстная коррекция на уровне слов, а не глобальная замена символов:

1. Если слово не содержит кириллицы — оно считается англоязычным и не изменяется.
2. Если слово не содержит латиницы — оно уже корректно написано и не изменяется.
3. Только если в одном слове одновременно присутствуют кириллица и латиница, латинские символы заменяются на визуально соответствующие кириллические аналоги.

Таким образом:
* исправляется псевдокириллица,
* сохраняются английские слова и бренды,
* исключаются ложные замены.

In [9]:
LAT_TO_RU = {
    "a": "а",
    "b": "в",
    "c": "с",
    "e": "е",
    "h": "н",
    "k": "к",
    "m": "м",
    "o": "о",
    "p": "р",
    "t": "т",
    "x": "х",
    "y": "у",
}

CYRILLIC_RE = re.compile(r"[а-яё]", re.IGNORECASE)
LATIN_RE = re.compile(r"[a-z]", re.IGNORECASE)

def fix_mixed_word(word: str) -> str:
    # если в слове нет кириллицы — это почти точно английское слово
    if not CYRILLIC_RE.search(word):
        return word

    # если нет латиницы — тоже ничего не делаем
    if not LATIN_RE.search(word):
        return word

    return "".join(LAT_TO_RU.get(ch, ch) for ch in word)

def fix_pseudo_cyrillic_text(text: str) -> str:
    words = text.split()
    fixed_words = [fix_mixed_word(w) for w in words]
    return " ".join(fixed_words)

def normalize_text(text: str) -> str:
    if not text:
        return ""

    text = text.lower()
    text = re.sub(r"[^a-zа-яё0-9\s]", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"\s+", " ", text).strip()
    text = fix_pseudo_cyrillic_text(text)

    return text

## Парсинг одного HTML-объявления Avito

Функция `parse_ad_from_html` выполняет разбор одного локально сохранённого HTML-файла объявления Avito и преобразует его в структурированный словарь признаков.

### Извлекаемые данные

В процессе парсинга извлекаются:
- заголовок объявления;
- текст описания (сырое и нормализованное);
- категории товара (2 уровня);
- хлебные крошки (breadcrumbs);
- характеристики товара;
- ссылки на изображения и их количество.

### Извлечение описания

HTML-структура описаний на Avito неоднородна, поэтому используется fallback-логика:

1. В первую очередь описание извлекается из блока  
   `data-marker="item-view/item-description"`.
2. Если блок отсутствует — описание извлекается из контейнера  
   `#bx_item-description`, при этом игнорируются заголовки интерфейса.

После извлечения текст нормализуется:
- приводится к нижнему регистру;
- очищается от спецсимволов;
- исправляется псевдокириллица при сохранении английских слов.

Сохраняются две версии:
- `description_raw` — очищенный текст;
- `description_clean` — нормализованный текст для модели.

### Категории и характеристики

Категории извлекаются из breadcrumbs позиционно, без словарей и жёстких названий.  
Характеристики извлекаются по DOM-паттерну «название → значение» и не предполагают фиксированный набор полей.

### Результат

Функция возвращает словарь, соответствующий одному объявлению.  
Далее выполняется пакетная обработка всех HTML-файлов с сохранением результатов в список `ads_data`.

In [10]:
def parse_ad_from_html(html_path: Path) -> dict:
    html_text = html_path.read_text(encoding="utf-8", errors="ignore")
    soup = BeautifulSoup(html_text, "html.parser")

    # ---------- TITLE ----------
    title = None
    h1 = soup.find("h1")
    if h1:
        title = h1.get_text(strip=True)

    # ---------- DESCRIPTION ----------
    description_raw = None
    description_clean = None
    
    def extract_description(soup):
        # 1. Новый редизайн 
        block = soup.find(attrs={"data-marker": "item-view/item-description"})
        if block:
            text = block.get_text("\n", strip=True)
            if text:
                return text
    
        # 2. Старый вариант — внутри bx_item-description, но БЕЗ h2
        container = soup.find(id="bx_item-description")
        if container:
            ps = container.find_all("p")
            texts = [p.get_text(strip=True) for p in ps if p.get_text(strip=True)]
            if texts:
                return "\n".join(texts)
    
        return None
    
    
    description_raw = extract_description(soup)
    if description_raw:
        description_clean = normalize_text(description_raw)

    # ---------- CATEGORIES ----------
    bc = soup.find(attrs={"data-marker": "breadcrumbs"})
    breadcrumbs = []
    if bc:
        breadcrumbs = [a.get_text(strip=True) for a in bc.find_all("a") if a.get_text(strip=True)]

    category_level_1 = None
    category_level_2 = None
    category_level_3 = None
    category_level_4 = None
    if len(breadcrumbs) >= 3:
        core = breadcrumbs[1:]  # убираем "Главная"
        if len(core) >= 3:
            category_level_1 = core[-4]
            category_level_2 = core[-3]
            category_level_3 = core[-2]
            category_level_4 = core[-1]

    # ---------- CHARACTERISTICS ----------
    characteristics = {}
    for li in soup.find_all("li"):
        p = li.find("p")
        if not p:
            continue

        spans = p.find_all("span", recursive=False)
        if not spans:
            continue

        label = spans[0].get_text(strip=True)
        if not label:
            continue

        full_text = p.get_text(" ", strip=True)
        value = full_text[len(label):].strip()
        value = value.lstrip(":").strip()

        if value:
            characteristics[label.rstrip(":")] = value

    # ---------- IMAGES ----------
    image_urls = extract_image_urls_from_html_file(html_path)
    images = select_high_quality_images(image_urls)

    return {
        "ad_id": html_path.stem,
        "title": title,
        "description_raw": description_raw,
        "description_clean": description_clean,
        "category_level_1": category_level_1,
        "category_level_2": category_level_2,
        "category_level_3": category_level_3,
        "category_level_4": category_level_4,
        "breadcrumbs_raw": breadcrumbs,
        "characteristics": characteristics,
        "images_count": len(images),
        "image_urls": [u for u, _ in images],
    }

ads_data = []

html_files = sorted(HTML_DIR.glob("*.html"))
print("HTML files:", len(html_files))

for html_path in tqdm(html_files):
    try:
        ad = parse_ad_from_html(html_path)
        ads_data.append(ad)
    except Exception as e:
        print(f"Error in {html_path.name}: {e}")

HTML files: 379


100%|████████████████████████████████████████████████████████████████████████████████| 379/379 [55:50<00:00,  8.84s/it]


In [11]:
with OUT_JSONL.open("w", encoding="utf-8") as f:
    for ad in ads_data:
        f.write(json.dumps(ad, ensure_ascii=False) + "\n")

print("Saved:", OUT_JSONL)

ads_data[0]

Saved: data\dataset_top_woman.jsonl


{'ad_id': '1926983478',
 'title': 'Джемпер HV Polo',
 'description_raw': '— джемпер HV Polo, очень мало использовали, размер M, маломерит, 1450 руб',
 'description_clean': 'джемпер hv polo очень мало использовали размер m маломерит 1450 руб',
 'category_level_1': '…',
 'category_level_2': 'Женская одежда',
 'category_level_3': 'Топы и футболки',
 'category_level_4': '46 (M)',
 'breadcrumbs_raw': ['Главная',
  '…',
  '…',
  'Женская одежда',
  'Топы и футболки',
  '46 (M)'],
 'characteristics': {'Состояние': 'Хорошее',
  'Предмет одежды': 'Поло',
  'Размер': '46 (M)',
  'Материал основной части': 'Акрил',
  'Цвет': 'Красный'},
 'images_count': 1,
 'image_urls': ['https://70.img.avito.st/image/1/1.d6wkjLa520USO1lIRoVYtNov3U-Qr83HnS_ZQZgl0Uc.bUtOwyqWKO22VU3gY0voPwmE2Fr7DDKWvoJHBT9-x-M']}

## Сохранение изображений объявлений

В данном блоке выполняется сохранение изображений объявлений на диск на основе ранее сформированного списка `ads_data`.

Для каждого объявления:
- используется идентификатор объявления (`ad_id`);
- создаётся отдельная директория в `IMG_DIR`;
- изображения загружаются по URL и сохраняются в формате JPEG.

Используются только изображения, прошедшие предварительную фильтрацию по качеству и дедупликацию.  
Ошибки загрузки отдельных изображений не прерывают выполнение пайплайна.

### Результат

Формируется иерархия каталогов вида:

IMG_DIR / ad_id / 00.jpg, 01.jpg, ...

Данная структура используется далее для формирования мультимодального датасета (текст + изображения).

In [12]:
def save_images_from_ads_data(ads_data, img_dir: Path):
    for ad in tqdm(ads_data):
        ad_id = ad["ad_id"]
        urls = ad.get("image_urls", [])

        if not urls:
            continue

        ad_img_dir = img_dir / ad_id
        ad_img_dir.mkdir(exist_ok=True)

        for i, url in enumerate(urls):
            try:
                img = download_image(url)
                if img:
                    img_path = ad_img_dir / f"{i:02d}.jpg"
                    img.save(img_path, format="JPEG", quality=95)
            except Exception as e:
                print(f"Image error in ad {ad_id}: {e}")

save_images_from_ads_data(ads_data, IMG_DIR)

100%|████████████████████████████████████████████████████████████████████████████████| 379/379 [09:25<00:00,  1.49s/it]
